In [ ]:
import json, os
from mpcontribs.client import load_client
from pymatgen import Composition, Structure, MPRester

In [ ]:
project = 'MnO2_phase_selection'
client = load_client('your-api-key-here')
mpr = MPRester()

Retrieve and update project info

In [ ]:
client.projects.get_entry(pk=project, _fields=['_all']).result()

In [ ]:
phase_names = {
    'beta': 'Pyrolusite',
    'gamma': 'Intergrowth',
    'ramsdellite': 'Ramsdellite',
    'alpha': 'Hollandite',
    'lambda': 'Spinel',
    'delta': 'Layered',
    'other': 'Other',
}
client.projects.update_entry(pk=project, project={'other': {'phase−names': phase_names}}).result()

Create contributions

In [ ]:
# mp_contrib_phases: data/MPContrib_formatted_entries.json
# hull_states: data/MPContrib_hull_entries.json
# mp_dup: data/MPExisting_MnO2_ids.json
# mp_cmp: data/MPComplete_MnO2_ids.json
data = {}
for fn in os.scandir('data'):
    with open(fn, 'r') as f:
        data[fn.name] = json.load(f)

In [ ]:
other = [
    ['LiMnO2', -3.064, 'Y', '--'], ['KMnO2', -2.222, 'Y', '--'],
    ['Ca0.5MnO2', -2.941, 'Y', '--'], ['Na0.5MnO2', -1.415, 'Y', '--']
]

In [ ]:
# clear everything out first
has_more = True
while has_more:
    contribs = client.contributions.get_entries(
        project=project, _fields=['id'], _limit=200
    ).result()
    print(contribs['total_count'])
    has_more = contribs['has_more']
    for idx, d in enumerate(contribs['data']):
        client.contributions.delete_entry(pk=d['id']).result()
        print(idx, d['id'], 'deleted')

is_public = True
for idx, hstate in enumerate(data['MPContrib_hull_entries.json']):
    contrib = {'project': project, 'is_public': is_public}
    phase = hstate['phase']
    composition = Composition.from_dict(hstate['c'])
    structure = Structure.from_dict(hstate['s'])
    mpids = mpr.find_structure(structure)
    comp = composition.get_integer_formula_and_factor()[0]
    contrib['identifier'] = mpids[0] if mpids else comp

    phase_name = phase_names[phase]
    phase_data = data['MPContrib_formatted_entries.json'].get(phase_name, other)
    if not phase_data:
        print('no data found for', composition, phase_name)
        continue

    for iv, values in enumerate(phase_data):
        if Composition(values[0]) == composition:
            contrib['data'] = {'GS': values[2], 'ΔH': f'{values[1]} eV/mol'}
            if not isinstance(values[3], str):
                contrib['data']['ΔH|hyd'] = f'{values[3]} eV/mol'
            break
    else:
        print('no data found for', composition, phase)
        continue
        
    label = '2018/02/16'
    contribs = client.contributions.get_entries(
        identifier=contrib['identifier'], _fields=['id', 'structures']
    ).result()
    if contribs['total_count'] > 0:
        cid = contribs['data'][0]['id']
        print(idx, contrib['identifier'], 'duplicate of', cid)
        sid = contribs['data'][0]['structures'].get(label)
        if sid:
            s = Structure.from_dict(client.structures.get_entry(pk=sid, _fields=['_all']).result())
            if s == structure:
                print(idx, contrib['identifier'], 'identical structures -> skip')
                continue
            else:
                print(idx, contrib['identifier'], 'matching but different structures -> TODO')
                break
        else:
            print(idx, contrib['identifier'], 'duplicate but structure missing -> add')
    else:
        cid = client.contributions.create_entry(contribution=contrib).result()['id']
        print(idx, contrib['identifier'], cid, 'created')

    sdct = {'contribution': cid, 'label': label, 'name': comp, 'is_public': is_public}
    sdct.update(structure.as_dict())
    sid = client.structures.create_entry(structure=sdct).result()['id']
    print(idx, 'structure', sid, 'created')